# Bike Store Data Quality Assessment
### This is to analyze the data from the Bike Store to confirm its quality.

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, Series

#reads excel workbook into workable python format
df_CustDem = pd.ExcelFile('KPMG_VI_Data_Set.xlsx').parse('CustomerDemographic', header=1, index_col=0)
df_Transactions = pd.ExcelFile('KPMG_VI_Data_Set.xlsx').parse('Transactions', header=1)
df_NewCust = pd.ExcelFile('KPMG_VI_Data_Set.xlsx').parse('NewCustomerList', header=1)
df_Address = pd.ExcelFile('KPMG_VI_Data_Set.xlsx').parse('CustomerAddress', header=1) 


## Review and Clean
### Customer Demographic DataFrame
    Explore Customer Demographic dataframe and note inconsistencies and points of interest. 

In [132]:
#dataframe values were previously listed as objects, covert_dtypes converts into correct data types
df_CustDem = df_CustDem.convert_dtypes()
df_CustDem.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1 to 4000
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           4000 non-null   string        
 1   last_name                            3875 non-null   string        
 2   gender                               4000 non-null   string        
 3   past_3_years_bike_related_purchases  4000 non-null   Int64         
 4   DOB                                  3913 non-null   datetime64[ns]
 5   job_title                            3494 non-null   string        
 6   job_industry_category                3344 non-null   string        
 7   wealth_segment                       4000 non-null   string        
 8   deceased_indicator                   4000 non-null   string        
 9   default                              3698 non-null   object        
 10  owns_car    

In [133]:
df_CustDem.max()

first_name                                          Zulema
gender                                                   U
past_3_years_bike_related_purchases                     99
DOB                                    2002-03-11 00:00:00
wealth_segment                               Mass Customer
deceased_indicator                                       Y
owns_car                                               Yes
dtype: object

In [134]:
df_CustDem.min()

first_name                                          Aarika
gender                                                   F
past_3_years_bike_related_purchases                      0
DOB                                    1843-12-21 00:00:00
wealth_segment                           Affluent Customer
deceased_indicator                                       N
owns_car                                                No
dtype: object

By reviewing the min and max of values in each column we can see problems with the data. 
For example: one customer had 99 bike purchases within the last three years. This would be cause for further exploration. 
The gender column also has an unusal value in it that requires further understanding.
In the min exploration we can see that one cutomer was born in 1843- more likely a mistake than a customer is 178 years old.  

In [135]:
df_CustDem['gender'].value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: Int64

88 rows have the value U in the gender column- need clarification if it stands for Unanswered or a nonbinary answer. 2 values also need to be standardized from F and M to Female and Male. 

In [136]:
df_CustDem['wealth_segment'].value_counts()

Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: Int64

This column might be significant to examine more closely. What are the parameters that determine a Mass Customer from and High Net Worth and Affluent Customer? 
It is also important to note that the Mass Customer is 50% of the customer base. But which customer category produces the most revenue for the business?

In [137]:
#Number of total bikes purchased by each group
purchased_group = df_CustDem.groupby('wealth_segment')
purchased_group['past_3_years_bike_related_purchases'].sum()

wealth_segment
Affluent Customer    47822
High Net Worth       48281
Mass Customer        99457
Name: past_3_years_bike_related_purchases, dtype: Int64

From these two explorations we can see that not only is the Mass Customer group the largest percentage of customers, they also acount for almost 50% of bike purchases in the last 3 years. 

In [138]:
#find amount of null values in each column
df_CustDem.isnull().sum()

first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

#### Customer Demographic Summary
From this we can tell that there are several inconsistaencies and missing values in the data. desicion will be to determine if we can remove those columns from analysis or input replacement data. 
We also determined that several values were entered incorrectly and nest steps could be to remove those rows from the data. 
However prelimerary conclsuions can be made that the Mass customer group is responsible for 50% of bike pruchases and its cutomer base. 

### Transaction Dataframe

In [139]:
df_Transactions = df_Transactions.convert_dtypes()
df_Transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  Int64         
 1   product_id               20000 non-null  Int64         
 2   customer_id              20000 non-null  Int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  Int64         
 5   order_status             20000 non-null  string        
 6   brand                    19803 non-null  string        
 7   product_line             19803 non-null  string        
 8   product_class            19803 non-null  string        
 9   product_size             19803 non-null  string        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

When looking at the meta data of this dataframe we can see the there are several missing values in 7 of the columns. 6 of those have the same number of missing values. We can also see that even though there are two columns referencing dates, only one is formatted in a datefime format. 
We will explore these issues more below.

In [140]:
df_Transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [141]:
df_Transactions.min()

transaction_id                        1
product_id                            0
customer_id                           1
transaction_date    2017-01-01 00:00:00
order_status                   Approved
list_price                        12.01
standard_cost                      7.21
dtype: object

In [142]:
df_Transactions.max()

transaction_id                    20000
product_id                          100
customer_id                        5034
transaction_date    2017-12-30 00:00:00
order_status                  Cancelled
list_price                      2091.47
standard_cost                   1759.85
dtype: object

First steps we can tell that there are 11 columns and of those, online_orders has the most null values. the remaining 6 columns with null values all have the same amount of 197. Such uniformity among all 6 columns causes further investigation. 
Next we check for outliers by looking at the min and max values of each column. Of immediate interest is the large differece between list price: lowest at $12.01 and max of $2,091.47 confirming accurancy of those amounts is neccessary to validate. 
Second if the date. The transaction data was to be for three months but the earliest is January 1, 2017 and the latest December 30, 2017. That is a year versus a three month review. 

In [143]:
df_Transactions['order_status'].value_counts()
df_Transactions['product_line'].value_counts()
df_Transactions['product_class'].value_counts()
df_Transactions['product_size'].value_counts()

medium    12990
large      3976
small      2837
Name: product_size, dtype: Int64

Reviewing some of the columns cotaining strings to confirmed values were uniform buy counting the unique values in each column. 

In [144]:
df_Transactions['transaction_date'].head()

0   2017-02-25
1   2017-05-21
2   2017-10-16
3   2017-08-31
4   2017-10-01
Name: transaction_date, dtype: datetime64[ns]

In [145]:
df_Transactions['product_first_sold_date'].head()

0    41245
1    41701
2    36361
3    36145
4    42226
Name: product_first_sold_date, dtype: Int64

In [146]:
df_Transactions['product_first_sold_date'] = df_Transactions['product_first_sold_date'].fillna(0).astype('datetime64[ns]', errors='ignore')
df_Transactions['product_first_sold_date'].head()

0   1970-01-01 00:00:00.000041245
1   1970-01-01 00:00:00.000041701
2   1970-01-01 00:00:00.000036361
3   1970-01-01 00:00:00.000036145
4   1970-01-01 00:00:00.000042226
Name: product_first_sold_date, dtype: datetime64[ns]

#### Transaction Dataframe Summary
There are 197 null values in 6 columns and one column has 360 null values. The decision needs to be made to eithe remove the rows or columns from analysis are replace the nulls with alternative values. 
There are two columns that should be in a datetime format however the product sold date column is an integer format.
Several efforts to convert the integers in Product_first_sold column to datetime objects were unsuccessful. However it should be noted that when accessing the data in excel the conversion can be made, but the inconsisentacy does raise the question of accuracy.

## Customer Address Dataframe

In [148]:
df_Address = df_Address.convert_dtypes()
df_Address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   Int64 
 1   address             3999 non-null   string
 2   postcode            3999 non-null   Int64 
 3   state               3999 non-null   string
 4   country             3999 non-null   string
 5   property_valuation  3999 non-null   Int64 
dtypes: Int64(3), string(3)
memory usage: 199.3 KB


At first glance this dataset looks to be complete, no null values and datatype appears to align correctly with the column names. Next we will see if there are any repeat values in the customer_id column and the address and postcode columns to confrim no customer was entered twice. We will also look into the property_valuation column more. 

In [161]:
df_Address['customer_id'].duplicated().value_counts()

False    3999
Name: customer_id, dtype: int64

In [223]:
df_Address['address'].duplicated().sum()
dup_address = df_Address[df_Address['address'].duplicated()]
print(dup_address)


      customer_id                  address  postcode state    country  \
2470         2475         3 Talisman Place      4017   QLD  Australia   
2980         2985  3 Mariners Cove Terrace      2216   NSW  Australia   
3535         3540   64 Macpherson Junction      4061   QLD  Australia   

      property_valuation  
2470                   5  
2980                  10  
3535                   8  


In [226]:
df_Address['property_valuation'].value_counts(ascending=False)

9     647
8     646
10    577
7     493
11    281
6     238
5     225
4     214
12    195
3     186
1     154
2     143
Name: property_valuation, dtype: Int64

#### Customer Address Dataframe Summary
Although there are no repeated customer ids, we do have three repeated addresses which could mean more than one customer at the same address or a duplicated address created for the same customer. 
upon investigation we were also able to see that property_valuation is broken up into 12 categories with the top valuations being 9, 8, and 10. 

## Data Quality Assessment Summary
Below is a summary of the quality assessment of each dataset. 

#### Customer Demographic dataset
When reviewing the min and max of values in each column we can see problems with the data. For example: one customer had 99 bike purchases within the last three years. This would be cause for further exploration. And in theDOB column we can see that one cutomer was born in 1843- more likely a mistake than a customer is 178 years old. The gender column has 88 rows have the value U. Does this stand for Unanswered or a nonbinary? Needs clarification. 2 values also need to be standardized from F and M to Female and Male. The welth segment column might be significant to examine more closely. What are the parameters that determine a Mass Customer from and High Net Worth and Affluent Customer? It is also important to note that the Mass Customer is 50% of the customer base. When reviewing wealth segment with bike purchases we can see that not only is the Mass Customer group the largest percentage of customers, they also acount for almost 50% of bike purchases in the last 3 years. 
#### Transaction dataset
When looking at the meta data of this dataset we can see the there are several missing values in 7 of the columns. 6 of those columns have 197 null values and the othe 360. We can also see that even though there are two columns referencing dates, only one is formatted in a datetime format. Several efforts were made to convert the integer values of the product_sold_date column to no success. Of the 11 columns, online_orders has the most null values. the remaining 6 columns with null values all have the same amount of 197. Such uniformity among all 6 columns causes further investigation. Next we check for outliers by looking at the min and max values of each column. Of immediate interest is the large differece between list price: lowest at  12.01 and max of 2,091.47 confirming accurancy of those amounts is neccessary to validate. Another thing to note is the transaction data was to be for three monthsm but the earliest is January 1, 2017 and the latest December 30, 2017. That is a year versus a three month review.
#### Customer Address dataset
At first glance this dataset looks to be complete, no null values and datatype appears to align correctly with the column names. Although there are no repeated customer ids, we do have three repeated addresses which could mean more than one customer at the same address or a duplicated address created for the same customer. Upon investigation we were also able to see that property_valuation is broken up into 12 categories with the top valuations being 9, 8, and 10.

#### Conclusion
What we can see from looking into the data is that there are some inconsistencies and discrepincies with the three datasets. In particular the customer demographic and transaction data sets. Several descions need to be made about whether to omit the rows and columns containing the large amount of null values before any significant analysis can be done. 